In [2]:
import os
from functions import * #TODO select only necessary functions
from classes import TextSummary
import collections
import json

In [3]:
#TODO make Command line argument
regenerate_entire_cache = True

#TODO Dry these paths, make relative, eventually move to a JSON file
vault = "/Users/joachimpfefferkorn/Obsidian/Main_Vault"
daily_notes_aggregated = "/Users/joachimpfefferkorn/Obsidian/Main_Vault/daily_notes.md"
footer_path = "/Users/joachimpfefferkorn/repos/daily_note_organizer/footer.md"
cache_folder = "/Users/joachimpfefferkorn/repos/daily_note_organizer/cache"


empty_tag = "$empty_note_summary$"

with open(footer_path, 'r') as footer_file:
    footer = footer_file.read() #TODO update footer

if not regenerate_entire_cache:
    print("💾 Reading in existing cache")
    with open(f"{cache_folder}/summarized_note_cache.json", 'r') as summarized_note_cache: #TODO pickle or JSON?
        unordered_note_summaries = json.loads(summarized_note_cache.read()) #MOVING LOGIC TO AFTER CACHE CREATION
else:
    print("🦎 Regenerating entire cache")
    unordered_note_summaries = {}

for note in os.listdir(vault): #TODO test this edge case
    if is_daily_note(note) and note not in unordered_note_summaries:
        print(f"🧩 Adding empty entry for {note} (these will be summarized later)")
        unordered_note_summaries[note] = empty_tag
note_summaries = collections.OrderedDict(sorted(unordered_note_summaries.items()))
print("📚 Note summary dictionary sorted")

iterator = 0
for note, summary in note_summaries.items():
    iterator += 1
    print(f"\n🦕 Checking note {iterator} out of {len(note_summaries)}")
    if summary == empty_tag:
        print(f"🧙 creating note summary for {note}")
        note_summary = create_note_summary(f"{vault}/{note}")
#        print("NOTE SUMMARY:, ", note_summary) #DEBUG, delete later
        note_summaries[note] = note_summary
    else:
        print("🕶️ Note summary already present")
    print("📑", note)
    print("📝 ", note_summaries[note])


🦎 Regenerating entire cache
🧩 Adding empty entry for 2024-06-24.md (these will be summarized later)
🧩 Adding empty entry for 2024-08-27.md (these will be summarized later)
🧩 Adding empty entry for 2024-04-30.md (these will be summarized later)
🧩 Adding empty entry for 2024-08-13.md (these will be summarized later)
🧩 Adding empty entry for 2024-04-04.md (these will be summarized later)
🧩 Adding empty entry for 2023-12-06.md (these will be summarized later)
🧩 Adding empty entry for 2023-10-12.md (these will be summarized later)
🧩 Adding empty entry for 2023-10-26.md (these will be summarized later)
🧩 Adding empty entry for 2023-12-12.md (these will be summarized later)
🧩 Adding empty entry for 2023-11-08.md (these will be summarized later)
🧩 Adding empty entry for 2024-06-14.md (these will be summarized later)
🧩 Adding empty entry for 2024-08-17.md (these will be summarized later)
🧩 Adding empty entry for 2024-08-23.md (these will be summarized later)
🧩 Adding empty entry for 2024-06-20.

/Users/joachimpfefferkorn/repos/daily_note_organizer/.venv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


🦋 Section is small enough!
🏄 Small enough section added to new_sections
🏭 Summarizing 2023-10-11.md
📇 Summarizing section 0 out of 1
🧼 cleaning summarized note for markdown formatting
NOTE SUMMARY:,  Evaluating a machine learning model is a daily task. It's a fourth order polynomial that fits the training data but fails to generalize to new examples.

📑 2023-10-11.md
📝  $empty_note_summary$

🦕 Checking note 2 out of 182
🧙 creating note summary for 2023-10-12.md

🦋 Section is small enough!
🏄 Small enough section added to new_sections
🏭 Summarizing 2023-10-12.md
📇 Summarizing section 0 out of 1
🧼 cleaning summarized note for markdown formatting
NOTE SUMMARY:,  In Math for Machine Learning Week 2, Multivariate Calculus, we learn about variables, constants, context, partial derivatives and the total derivative.

📑 2023-10-12.md
📝  $empty_note_summary$

🦕 Checking note 3 out of 182
🧙 creating note summary for 2023-10-17.md



Token indices sequence length is longer than the specified maximum sequence length for this model (2704 > 1024). Running this sequence through the model will result in indexing errors


🐘 Original section must be split by 2! Section tokens: 2704 Max model length: 1024


Token indices sequence length is longer than the specified maximum sequence length for this model (1439 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1266 > 1024). Running this sequence through the model will result in indexing errors


🪸 New sections updated with split subsections, 🔃 recursively split is recursing
⚔️ Splitting subsection 0
🐘 Original section must be split by 3! Section tokens: 1439 Max model length: 1024
🪸 New sections updated with split subsections, 🔃 recursively split is recursing
⚔️ Splitting subsection 0
🦋 Section is small enough!
🏄 Small enough section added to new_sections
⚔️ Splitting subsection 1
🦋 Section is small enough!
🏄 Small enough section added to new_sections
⚔️ Splitting subsection 2
🦋 Section is small enough!
🏄 Small enough section added to new_sections
⚔️ Splitting subsection 1
🐘 Original section must be split by 3! Section tokens: 1266 Max model length: 1024
🪸 New sections updated with split subsections, 🔃 recursively split is recursing
⚔️ Splitting subsection 0
🦋 Section is small enough!
🏄 Small enough section added to new_sections
⚔️ Splitting subsection 1
🦋 Section is small enough!
🏄 Small enough section added to new_sections
⚔️ Splitting subsection 2
🦋 Section is small enough!

KeyboardInterrupt: 

In [ ]:
if regenerate_entire_cache:
    print("💿 Saving cache")
    json_cache = json.dumps(note_summaries, sort_keys=True, indent=4)
    with open(f"{cache_folder}/summarized_note_cache.json", 'w') as summarized_note_cache:
        summarized_note_cache.write(json_cache)

print("💾 Reading in newly updated cache")
with open(f"{cache_folder}/summarized_note_cache.json", 'r') as summarized_note_cache:
    note_summary_dict = json.loads(summarized_note_cache.read())

In [ ]:
#INIT
years = []
months = []

with open(daily_notes_aggregated, 'w') as dailynote_file:
    for note in note_summary_dict:
        if note[0:4] not in years:
            print(f"🗓️ Adding {note[0:4]}")
            dailynote_file.write("# {year}\n".format(year = note[0:4]))
            years.append(note[0:4])
        if note[0:7] not in months:
            month = month_writer(note)
            print(f"Adding {month}")
            dailynote_file.write("## {month}\n".format(month = month))
            months.append(note[0:7])

        note_link = "[[{note_link}]]".format(note_link = note[:-3])
        note_path = os.path.join(vault, note)
        dailynote_file.write(note_link)
        print("📑 Note Added: ", note_link)
        dailynote_file.write("\n - ")
        dailynote_file.write(note_summary_dict[note])
        print(f"📝 Summary Added:\n{note_summary_dict[note]}")
        dailynote_file.write("\n")
